# 02 - Convolutional Neural Network

## Import Statements

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

## Loading the datasets

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('../MNIST', one_hot=True)

Extracting MNIST/train-images-idx3-ubyte.gz
Extracting MNIST/train-labels-idx1-ubyte.gz
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz


In [3]:
print('Length of training sets:', len(data.train.labels))
print('Length of testing sets:', len(data.test.labels))
print('Length of validation sets:', len(data.validation.labels))

Length of training sets: 55000
Length of testing sets: 10000
Length of validation sets: 5000


## Define Hyperparameters

In [4]:
image_size = 28
image_shape = (image_size, image_size)
image_shape_flat = image_size * image_size
num_classes = 10
learning_rate = 1e-4
batch_size = 100
dropout = 1.0

## Define some helper functions

In [5]:
# Weights initialization
def weight(shape):
    initial = tf.truncated_normal(shape, stddev=0.04)
    return tf.Variable(initial)

# Bias initialization
def bias(shape):
    initial = tf.constant(0.04, shape=shape)
    return tf.Variable(initial)

# Convolutional Operation
def conv2D(X, W):
    return tf.nn.conv2d(X, W, strides=[1,1,1,1], padding='SAME')

# Down-sampling or Max Pooling
def max_pool(X):
    return tf.nn.max_pool(X, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

# Flatten 4D tensor to 2D tensor
def flatten(layer):
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:].num_elements()
    return tf.reshape(layer, [-1, num_features])

## Define Placeholder variables

In [6]:
X = tf.placeholder(tf.float32, [None, image_shape_flat])
y = tf.placeholder(tf.float32, [None, num_classes])
y_true = tf.argmax(y, 1)
keep_prob = tf.placeholder(tf.float32)

## 1st Convolutional Layer

In [7]:
# Reshape X to a 4D tensor [num_images, width, height, channel]
X_image = tf.reshape(X, [-1, image_size, image_size, 1])
W_conv1 = weight([5,5,1,32])
b_conv1 = bias([32])
h_conv1 = tf.nn.relu(conv2D(X_image, W_conv1) + b_conv1)
h_pool1 = max_pool(h_conv1)

In [8]:
h_pool1

<tf.Tensor 'MaxPool:0' shape=(?, 14, 14, 32) dtype=float32>

## 2nd Convolutional Layer

In [9]:
W_conv2 = weight([5, 5, 32, 64])
b_conv2 = bias([64])

h_conv2 = tf.nn.relu(conv2D(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool(h_conv2)

In [10]:
h_conv2

<tf.Tensor 'Relu_1:0' shape=(?, 14, 14, 64) dtype=float32>

In [11]:
h_pool2

<tf.Tensor 'MaxPool_1:0' shape=(?, 7, 7, 64) dtype=float32>

## Fully Connected Layer

In [12]:
h_pool2_flat = flatten(h_pool2)
W_fc1 = weight([7*7*64, 1024])
b_fc1 = bias([1024])

h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [13]:
h_fc1

<tf.Tensor 'Relu_2:0' shape=(?, 1024) dtype=float32>

## Dropout Layer

In [14]:
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [15]:
h_fc1_drop

<tf.Tensor 'dropout/mul:0' shape=(?, 1024) dtype=float32>

## Output Layer

In [16]:
W_out = weight([1024, num_classes])
b_out = bias([num_classes])

y_pred = tf.matmul(h_fc1_drop, W_out) + b_out
y_pred_true = tf.argmax(y_pred, 1)

In [17]:
y_pred

<tf.Tensor 'add_3:0' shape=(?, 10) dtype=float32>

## Evaluate Model's error using \``softmax_cross_entropy_with_logits`\`

In [18]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y)
cost = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

## Starting the Tensorflow `InteractiveSession`

In [19]:
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()
sess.run(init)

## Evaluating Accuracy

In [20]:
correct = tf.equal(y_true, y_pred_true)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

## Training the Convolutional Neural Network Model

In [21]:
from tqdm import tqdm
def optimize(num_iter=100):
    for i in tqdm(range(num_iter)):
        batch_X, batch_y = data.train.next_batch(batch_size)
        sess.run(optimizer, feed_dict={X:batch_X, y:batch_y, keep_prob: dropout})

In [24]:
def print_accuracy():
    acc = accuracy.eval(feed_dict={X: data.validation.images,
                                        y: data.validation.labels,
                                        keep_prob: dropout})
    print('Accuracy = {:.1%}'.format(acc))

## Optimization Iterations

In [23]:
optimize(1) # 1 Iteration

100%|██████████| 1/1 [00:00<00:00,  1.40it/s]


In [25]:
print_accuracy()

Accuracy = 25.2%


In [26]:
optimize(9) # 10 Iterations ( 1+9 )

100%|██████████| 9/9 [00:03<00:00,  2.83it/s]


In [27]:
print_accuracy()

Accuracy = 45.9%


In [28]:
optimize(90) # 100 Iterations ( 1+9+90 )

100%|██████████| 90/90 [00:31<00:00,  3.05it/s]


In [29]:
print_accuracy()

Accuracy = 85.9%


In [30]:
optimize(900) # 1,000 Iterations ( 1+9+90+900 )

100%|██████████| 900/900 [05:11<00:00,  3.06it/s]


In [31]:
print_accuracy()

Accuracy = 97.5%


In [32]:
optimize(1000) # 2,000 Iterations ( 1+9+90+900+1000 )

100%|██████████| 1000/1000 [06:02<00:00,  2.99it/s]


In [33]:
print_accuracy()

Accuracy = 98.4%


In [34]:
optimize(8000) # 10,000 Iterations ( 1+9+90+900+1000+8000 )

100%|██████████| 8000/8000 [1:26:21<00:00,  2.90it/s]     


In [35]:
print_accuracy()

Accuracy = 99.1%


In [ ]:
sess.close()